# Classify snow-covered area (SCA) in Sentinel-2, Landsat 8/9, and PlanetScope imagery: full pipelines

Rainey Aberle

Department of Geosciences, Boise State University

2022

### Requirements:
- Area of Interest (AOI) shapefile: where snow will be classified in all available images. 
- Google Earth Engine (GEE) account: used to pull DEM over the AOI. Sign up for a free account [here](https://earthengine.google.com/new_signup/). 
- Digital elevation model (DEM) (_optional_): used to extract elevations over the AOI and for each snowline. If no DEM is provided, the ASTER Global DEM will be loaded through GEE. 

### Outline:
__0. Setup__ paths in directory, file locations, authenticate GEE - _modify this section!_

__1. Sentinel-2 Top of Atmosphere (TOA) imagery:__ full pipeline

__2. Sentinel-2 Surface Reflectance (SR) imagery:__ full pipeline

__3. Landsat 8/9 Surface Reflectance (SR) imagery:__ full pipeline

__4. PlanetScope Surface Reflectance (SR) imagery:__ full pipeline

-------


### 0. Setup

#### Define paths in directory and desired settings. 
Modify lines located within the following:

`#### MODIFY HERE ####`  

`#####################`

In [ ]:
##### MODIFY HERE #####

# -----Paths in directory
site_name = 'Wolverine'
# path to snow-cover-mapping/ - Make sure you include a "/" at the end
base_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/snow-cover-mapping/'
# path to AOI including the name of the shapefile
AOI_path = base_path + '../study-sites/' + site_name + '/AOIs/'
# AOI file name
AOI_fn =  'Wolverine_USGS_glacier_outline_2020.shp'
# path to DEM including the name of the tif file
# Note: set DEM_path==None and DEM_fn=None if you want to use the ASTER GDEM via Google Earth Engine
DEM_path = base_path + '../study-sites/' + site_name + '/DEMs/'
# DEM file name
DEM_fn = 'Wolverine_20200502_DEM_filled.tif'
# path for output images
out_path = base_path + '../study-sites/' + site_name + '/imagery/'
# path to PlanetScope images
# Note: set PS_im_path=None if not using PlanetScope
PS_im_path = out_path + 'PlanetScope/raw_images/'
# path for output figures
figures_out_path = base_path + '../study-sites/' + site_name + '/figures/'

# -----Define image search filters
date_start = '2013-05-01'
date_end = '2023-01-01'
month_start = 5
month_end = 11
cloud_cover_max = 100

# -----Determine settings
plot_results = True # = True to plot figures of results for each image where applicable
skip_clipped = False # = True to skip images where bands appear "clipped", i.e. max(blue) < 0.8
crop_to_AOI = True # = True to crop images to AOI before calculating SCA
save_outputs = True # = True to save SCAs and snowlines to file
save_figures = True # = True to save SCA output figures to file

#######################

# -----Import packages
import xarray as xr
import os
import numpy as np
import glob
from matplotlib import pyplot as plt, dates
import matplotlib
import rasterio as rio
import geopandas as gpd
import pandas as pd
import sys
import ee
import pickle
import time

# -----Set paths for output files
PS_im_masked_path = out_path + 'PlanetScope/masked/'
PS_im_mosaics_path = out_path + 'PlanetScope/mosaics/'
im_classified_path = out_path + 'classified/'
snowlines_path = out_path + 'snowlines/'

# -----Add path to functions
sys.path.insert(1, base_path+'functions/')
import pipeline_utils as f

# -----Load dataset dictionary
with open(base_path + 'inputs-outputs/datasets_characteristics.pkl', 'rb') as fn:
    dataset_dict = pickle.load(fn)

#### Authenticate and initialize Google Earth Engine (GEE). 

__Note:__ The first time you run the following cell, you will be asked to authenticate your GEE account for use in this notebook. This will send you to an external web page, where you will walk through the GEE authentication workflow and copy an authentication code back into the space below this cell when prompted. 

In [ ]:
try:
    ee.Initialize()
except: 
    ee.Authenticate()
    ee.Initialize()

#### Load AOI and DEM

In [ ]:
# -----Load AOI as gpd.GeoDataFrame
AOI = gpd.read_file(AOI_path + AOI_fn)
# reproject the AOI to WGS to solve for the optimal UTM zone
AOI_WGS = AOI.to_crs(4326)
AOI_WGS_centroid = [AOI_WGS.geometry[0].centroid.xy[0][0],
                    AOI_WGS.geometry[0].centroid.xy[1][0]]
# grab the optimal UTM zone EPSG code
epsg_UTM = f.convert_wgs_to_utm(AOI_WGS_centroid[0], AOI_WGS_centroid[1])
    
# -----Load DEM as Xarray DataSet
if DEM_fn==None:
    # query GEE for DEM
    DEM, AOI_UTM = f.query_GEE_for_DEM(AOI)
else:
    # reproject AOI to UTM
    AOI_UTM = AOI.to_crs('EPSG:'+str(epsg_UTM))
    # load DEM as xarray DataSet
    DEM = xr.open_dataset(DEM_path + DEM_fn)
    DEM = DEM.rename({'band_data': 'elevation'})
    # reproject the DEM to the optimal UTM zone
    DEM = DEM.rio.reproject(str('EPSG:'+epsg_UTM))

## 1. Sentinel-2 TOA imagery

In [ ]:
print('Sentinel-2 TOA')
print('----------')

# -----Load trained classifier and feature columns
clf_fn = base_path+'inputs-outputs/S2_TOA_classifier_all_sites.sav'
clf = pickle.load(open(clf_fn, 'rb'))
feature_cols_fn = base_path+'inputs-outputs/S2_TOA_feature_cols.pkl'
feature_cols = pickle.load(open(feature_cols_fn,'rb'))

# -----Query GEE for imagery
dataset = 'Sentinel2_TOA'
im_list = f.query_GEE_for_Sentinel2(dataset, dataset_dict, site_name, 
                                       AOI_UTM, date_start, date_end, month_start, 
                                       month_end, cloud_cover_max)
im_list_size = im_list.size().getInfo()

# -----Loop through images
if im_list_size==0: # check that images were found
    print('No images found to classify, quiting...')
else:
    
    for i in range(0, im_list_size):
        
        # -----Select image by index
        im = ee.Image(ee.List(im_list).get(i))
        # get image time
        im_date = im.date().format(None, 'GMT').getInfo()
        print(' ')
        print(str(i+1)+'/'+str(im_list_size))
        print(im_date)
                
            
        # -----Check if classified image and snowline already exists in file
        im_classified_fn = im_date.replace('-','').replace(':','') + '_' + site_name + '_' + dataset + '_classified.nc'
        snowline_fn = im_date.replace('-','').replace(':','') + '_' + site_name + '_' + dataset + '_snowline.pkl'
        if os.path.exists(im_classified_path + im_classified_fn) & os.path.exists(snowlines_path + snowline_fn):
            
            print('Classified image already exists in file, loading...')
            im_classified = xr.open_dataset(im_classified_path + im_classified_fn)
            print('Snowline already exists in file, loading...')
            snowline_df = pd.read_pickle(snowlines_path + snowline_fn)
        
        else:  
            # -----Convert image to xarray.Dataset
            if site_name=='Gulkana':
                res = 12
            else:
                res = dataset_dict[dataset]['resolution_m']
            im_xr = im.wx.to_xarray(scale=res, crs='EPSG:4326')
            # reproject to UTM CRS
            im_xr_UTM = im_xr.rio.reproject('EPSG:'+epsg_UTM)
            # replace no data values with NaN and account for image scalar
            bands = [band for band in dataset_dict[dataset]['bands'] if 'QA' not in band]
            for band in bands:
                im_xr_UTM[band] = xr.where(im_xr_UTM[band] != dataset_dict[dataset]['no_data_value'],
                                           im_xr_UTM[band] / dataset_dict[dataset]['im_scalar'], np.nan)

            # -----Add NDSI band
            im_xr_UTM['NDSI'] = ((im_xr_UTM[dataset_dict[dataset]['NDSI'][0]] - im_xr_UTM[dataset_dict[dataset]['NDSI'][1]]) 
                                 / (im_xr_UTM[dataset_dict[dataset]['NDSI'][0]] + im_xr_UTM[dataset_dict[dataset]['NDSI'][1]]))

            # -----Classify image
            im_classified = f.classify_image(im_xr_UTM, clf, feature_cols, crop_to_AOI, 
                                             AOI_UTM, dataset, dataset_dict, site_name, 
                                             im_classified_fn, im_classified_path)
            if type(im_classified)==str:
                continue
        
            # -----Delineate snowline(s)
            plot_results = True
            snowline_df = f.delineate_im_snowline(im_xr_UTM, im_classified, site_name, AOI_UTM, DEM, 
                                                  dataset_dict, dataset, im_date, snowline_fn, 
                                                  snowlines_path, figures_out_path, plot_results)
            
        print('Median snowline elevation: ' + str(snowline_df['snowlines_elevs_median'][0]) + ' m')

## 2. Sentinel-2 SR imagery

In [ ]:
# -----Load trained classifier and feature columns
clf_fn = base_path+'inputs-outputs/S2_SR_classifier_all_sites.sav'
clf = pickle.load(open(clf_fn, 'rb'))
feature_cols_fn = base_path+'inputs-outputs/S2_SR_feature_cols.pkl'
feature_cols = pickle.load(open(feature_cols_fn,'rb'))

# -----Query GEE for imagery
dataset = 'Sentinel2_SR'
im_list = f.query_GEE_for_Sentinel2(dataset, dataset_dict, site_name, 
                                       AOI_UTM, date_start, date_end, month_start, 
                                       month_end, cloud_cover_max)
im_list_size = im_list.size().getInfo()

# -----Loop through images
if im_list_size==0: # check that images were found
    print('No images found to classify, quiting...')
else:
    
    for i in range(0, im_list_size):
        
        # -----Select image by index
        im = ee.Image(ee.List(im_list).get(i))
        # get image time
        im_date = im.date().format(None, 'GMT').getInfo()
        print(' ')
        print(str(i+1)+'/'+str(im_list_size))
        print(im_date)
                
            
        # -----Check if classified image and snowline already exists in file
        im_classified_fn = im_date.replace('-','').replace(':','') + '_' + site_name + '_' + dataset + '_classified.nc'
        snowline_fn = im_date.replace('-','').replace(':','') + '_' + site_name + '_' + dataset + '_snowline.pkl'
        if os.path.exists(im_classified_path + im_classified_fn) & os.path.exists(snowlines_path + snowline_fn):
            
            print('Classified image already exists in file, loading...')
            im_classified = xr.open_dataset(im_classified_path + im_classified_fn)
            print('Snowline already exists in file, loading...')
            snowline_df = pd.read_pickle(snowlines_path + snowline_fn)
        
        else:  
            # -----Convert image to xarray.Dataset
            if site_name=='Gulkana':
                res = 12
            else:
                res = dataset_dict[dataset]['resolution_m']
            im_xr = im.wx.to_xarray(scale=res, crs='EPSG:4326')
            # reproject to UTM CRS
            im_xr_UTM = im_xr.rio.reproject('EPSG:'+epsg_UTM)
            # replace no data values with NaN and account for image scalar
            bands = [band for band in dataset_dict[dataset]['bands'] if 'QA' not in band]
            for band in bands:
                im_xr_UTM[band] = xr.where(im_xr_UTM[band] != dataset_dict[dataset]['no_data_value'],
                                           im_xr_UTM[band] / dataset_dict[dataset]['im_scalar'], np.nan)

            # -----Add NDSI band
            im_xr_UTM['NDSI'] = ((im_xr_UTM[dataset_dict[dataset]['NDSI'][0]] - im_xr_UTM[dataset_dict[dataset]['NDSI'][1]]) 
                                 / (im_xr_UTM[dataset_dict[dataset]['NDSI'][0]] + im_xr_UTM[dataset_dict[dataset]['NDSI'][1]]))

            # -----Classify image
            im_classified = f.classify_image(im_xr_UTM, clf, feature_cols, crop_to_AOI, 
                                             AOI_UTM, dataset, dataset_dict, site_name, 
                                             im_classified_fn, im_classified_path)
            if type(im_classified)==str:
                continue
        
            # -----Delineate snowline(s)
            plot_results = True
            snowline_df = f.delineate_im_snowline(im_xr_UTM, im_classified, site_name, AOI_UTM, DEM, 
                                                  dataset_dict, dataset, im_date, snowline_fn, 
                                                  snowlines_path, figures_out_path, plot_results)
            
        print('Median snowline elevation: ' + str(snowline_df['snowlines_elevs_median'][0]) + ' m')

## 3. Landsat 8/9 SR

In [ ]:
# -----Load trained classifier and feature columns
clf_fn = base_path+'inputs-outputs/L_classifier_all_sites.sav'
clf = pickle.load(open(clf_fn, 'rb'))
feature_cols_fn = base_path+'inputs-outputs/L_feature_cols.pkl'
feature_cols = pickle.load(open(feature_cols_fn,'rb'))

# -----Query GEE for imagery
dataset = 'Landsat'
im_list = f.query_GEE_for_Landsat_SR(AOI_UTM, date_start, date_end, month_start, month_end, 
                                    cloud_cover_max, site_name, dataset, dataset_dict, out_path)
im_list_size = im_list.size().getInfo()

# -----Loop through images
if im_list_size==0: # check that images were found
    print('No images found to classify, quiting...')
else:
    
    for i in range(0, im_list_size):
        
        # -----Select image by index
        im = ee.Image(ee.List(im_list).get(i))
        # get image time
        im_date = im.date().format(None, 'GMT').getInfo()
        print(' ')
        print(str(i+1)+'/'+str(im_list_size))
        print(im_date)
                
            
        # -----Check if classified image and snowline already exists in file
        im_classified_fn = im_date.replace('-','').replace(':','') + '_' + site_name + '_' + dataset + '_classified.nc'
        snowline_fn = im_date.replace('-','').replace(':','') + '_' + site_name + '_' + dataset + '_snowline.pkl'
        if os.path.exists(im_classified_path + im_classified_fn) & os.path.exists(snowlines_path + snowline_fn):
            
            print('Classified image already exists in file, loading...')
            im_classified = xr.open_dataset(im_classified_path + im_classified_fn)
            print('Snowline already exists in file, loading...')
            snowline_df = pd.read_pickle(snowlines_path + snowline_fn)
        
        else:  
            # -----Convert image to xarray.Dataset
            res = dataset_dict[dataset]['resolution_m']
            im_xr = im.wx.to_xarray(scale=res, crs='EPSG:4326')
            # reproject to UTM CRS
            im_xr_UTM = im_xr.rio.reproject('EPSG:'+epsg_UTM)
            # replace no data values with NaN and account for image scalar
            bands = [band for band in dataset_dict[dataset]['bands'] if 'QA' not in band]
            for band in bands:
                im_xr_UTM[band] = xr.where(im_xr_UTM[band] != dataset_dict[dataset]['no_data_value'],
                                           im_xr_UTM[band] / dataset_dict[dataset]['im_scalar'], np.nan)

            # -----Add NDSI band
            im_xr_UTM['NDSI'] = ((im_xr_UTM[dataset_dict[dataset]['NDSI'][0]] - im_xr_UTM[dataset_dict[dataset]['NDSI'][1]]) 
                                 / (im_xr_UTM[dataset_dict[dataset]['NDSI'][0]] + im_xr_UTM[dataset_dict[dataset]['NDSI'][1]]))

            # -----Classify image
            im_classified = f.classify_image(im_xr_UTM, clf, feature_cols, crop_to_AOI, 
                                             AOI_UTM, dataset, dataset_dict, site_name, 
                                             im_classified_fn, im_classified_path)
            if type(im_classified)==str:
                continue
        
            # -----Delineate snowline(s)
            plot_results = True
            snowline_df = f.delineate_im_snowline(im_xr_UTM, im_classified, site_name, AOI_UTM, DEM, 
                                                  dataset_dict, dataset, im_date, snowline_fn, 
                                                  snowlines_path, figures_out_path, plot_results)
            
        print('Median snowline elevation: ' + str(snowline_df['snowlines_elevs_median'][0]) + ' m')

## 4. PlanetScope SR

In [ ]:
# -----Load trained classifier and feature columns
clf_fn = base_path+'inputs-outputs/PS_classifier_all_sites.sav'
clf = pickle.load(open(clf_fn, 'rb'))
feature_cols_fn = base_path+'inputs-outputs/PS_feature_cols.pkl'
feature_cols = pickle.load(open(feature_cols_fn,'rb'))
dataset = 'PlanetScope'

# -----Read surface reflectance file names
os.chdir(PS_im_path)
im_fns = glob.glob('*SR*.tif')
im_fns = sorted(im_fns) # sort chronologically
plot_results = False

# ----Mask clouds and cloud shadows in all images
print('Masking images using cloud bitmask...')
for i, im_fn in enumerate(im_fns):
    f.PS_mask_im_pixels(PS_im_path, im_fn, PS_im_masked_path, save_outputs, plot_results)

# -----Mosaic images captured within same hour
print('Mosaicking images captured in the same hour...')
# read masked image file names
os.chdir(PS_im_masked_path)
im_masked_fns = glob.glob('*_mask.tif')
im_masked_fns = sorted(im_masked_fns) # sort chronologically
# mosaic images by date
f.PS_mosaic_ims_by_date(PS_im_masked_path, im_masked_fns, PS_im_mosaics_path, AOI_UTM, plot_results)
print(' ')

# -----Adjust image radiometry
# read mosaicked image file names
os.chdir(PS_im_mosaics_path)
im_mosaic_fns = glob.glob('*.tif')
im_mosaic_fns = sorted(im_mosaic_fns)
# create a polygon(s) of the top 20th percentile elevations within the AOI
plot_results=False 
polygon_top, polygon_bottom, im_mosaic_fn, im_mosaic = f.create_AOI_elev_polys(AOI_UTM, PS_im_mosaics_path, im_mosaic_fns, DEM)
# loop through images
for i, im_mosaic_fn in enumerate(im_mosaic_fns):
    print(' ')
    print(str(i+1)+'/'+str(len(im_mosaic_fns)))

    # adjust radiometry
    im_adj, im_adj_method = f.PS_adjust_image_radiometry(im_mosaic_fn, PS_im_mosaics_path, polygon_top, 
                                                         polygon_bottom, AOI_UTM, dataset_dict, dataset, 
                                                         site_name, skip_clipped, plot_results)
    if type(im_adj)==str: # skip if there was an error in adjustment
        continue
    
    # -----Determiine image date
    im_date = im_mosaic_fn[0:8] + 'T' + im_mosaic_fn[9:11] + ':00:00'
    
    # -----Classify image
    im_classified_fn = im_date.replace('-','').replace(':','') + '_' + site_name + '_' + dataset + '_classified.nc'
    if os.path.exists(im_classified_path + im_classified_fn):
        print('Classified image already exists in file, loading...')
        im_classified = xr.open_dataset(im_classified_path + im_classified_fn)
    else:
        im_classified = f.classify_image(im_adj, clf, feature_cols, crop_to_AOI, 
                                         AOI_UTM, dataset, dataset_dict, site_name, 
                                         im_classified_fn, im_classified_path)

    if type(im_classified)==str:
        continue    
    
    # -----Delineate snowline(s)
    plot_results=True
    snowline_fn = im_date.replace('-','').replace(':','') + '_' + site_name + '_' + dataset + '_snowline.pkl'
    if os.path.exists(snowlines_path + snowline_fn):
        print('Snowline already exists in file, loading...')
        snowline_df = pd.read_pickle(snowlines_path + snowline_fn)
    else:
        snowline_df = f.delineate_im_snowline(im_adj, im_classified, site_name, AOI_UTM, DEM, 
                                              dataset_dict, dataset, im_date, snowline_fn, 
                                              snowlines_path, figures_out_path, plot_results)
    print('Median snowline elevation: ' + str(snowline_df['snowlines_elevs_median'][0]) + ' m')

## _Optional:_ Compile snowlines into single file

## _Optional:_ Compile figures into single .gif file

In [ ]:
from shapely.geometry import shape, Polygon, MultiPolygon, Point
def plot_snowline_image(im, im_classified, snowline_df, AOI, DEM, site_name, im_date, figures_out_path, dataset, dataset_dict):

    # -----Make directory for snowlines (if it does not already exist in file)
    if os.path.exists(figures_out_path)==False:
        os.mkdir(figures_out_path)
        print("Made directory for figures:" + figures_out_path)

    # -----Subset dataset_dict to dataset
    ds_dict = dataset_dict[dataset]

    # -----Define image bands
    bands = [x for x in im.data_vars]
    bands = [band for band in bands if 'QA' not in band]

    # -----Remove time dimension
    im = im.isel(time=0)
    im_classified = im_classified.isel(time=0)

    # -----Create no data mask
    no_data_mask = xr.where(np.isnan(im_classified), 1, 0).to_array().data[0]
    # convert to polygons
    no_data_polygons = []
    for s, value in rio.features.shapes(no_data_mask.astype(np.int16),
                                        mask=(no_data_mask > 0),
                                        transform=im.rio.transform()):
        no_data_polygons.append(shape(s))
    no_data_polygons = MultiPolygon(no_data_polygons)

    # -----Mask the DEM using the AOI
    # create AOI mask
    mask_AOI = rio.features.geometry_mask(AOI.geometry,
                                      out_shape=(len(DEM.y), len(DEM.x)),
                                      transform=DEM.rio.transform(),
                                      invert=True)
    # convert mask to xarray DataArray
    mask_AOI = xr.DataArray(mask_AOI , dims=("y", "x"))
    # mask DEM values outside the AOI
    DEM_AOI = DEM.copy(deep=True)
    DEM_AOI['elevation'].data = np.where(mask_AOI==True, DEM_AOI['elevation'].data, np.nan)

    # -----Interpolate DEM to the image coordinates
    DEM_AOI_interp = DEM_AOI.interp(x=im_classified.x.data,
                                    y=im_classified.y.data,
                                    method="nearest")

    # -----Determine snow covered elevations
    # create array of elevation for all un-masked pixels
    all_elev = np.ravel(np.where(~np.isnan(im_classified.classified.data), DEM_AOI_interp.elevation.data, np.nan))
    all_elev = all_elev[~np.isnan(all_elev)] # remove NaNs
    # create array of snow-covered pixel elevations
    snow_est_elev = np.ravel(np.where(im_classified.classified.data <= 2, DEM_AOI_interp.elevation.data, np.nan))
    snow_est_elev = snow_est_elev[~np.isnan(snow_est_elev)] # remove NaNs

    # -----Create elevation histograms
    # determine bins to use in histograms
    elev_min = np.fix(np.nanmin(DEM_AOI_interp.elevation.data.flatten())/10)*10
    elev_max = np.round(np.nanmax(DEM_AOI_interp.elevation.data.flatten())/10)*10
    bin_edges = np.linspace(elev_min, elev_max, num=int((elev_max-elev_min)/10 + 1))
    bin_centers = (bin_edges[1:] + bin_edges[0:-1]) / 2
    # calculate elevation histograms
    H_elev = np.histogram(all_elev, bins=bin_edges)[0]
    H_snow_est_elev = np.histogram(snow_est_elev, bins=bin_edges)[0]
    H_snow_est_elev_norm = H_snow_est_elev / H_elev

    # -----Make all pixels at elevations >75% snow coverage = snow
    # determine elevation with > 75% snow coverage
    if len(np.where(H_snow_est_elev_norm > 0.75)[0]) > 1:
        elev_75_snow = bin_centers[np.where(H_snow_est_elev_norm > 0.75)[0][0]]
        # set all pixels above the elev_75_snow to snow (1)
        im_classified_adj = xr.where(DEM_AOI_interp['elevation'].isel(band=0) > elev_75_snow, 1, im_classified) # set all values above elev_75_snow to snow (1)
        im_classified_adj = im_classified_adj.squeeze(drop=True) # drop unecessary dimensions
        H_snow_est_elev_norm[bin_centers >= elev_75_snow] = 1
    else:
        im_classified_adj = im_classified.squeeze(drop=True)

    # -----Plot results
    fig, ax = plt.subplots(2, 2, figsize=(12,8), gridspec_kw={'height_ratios': [3, 1]})
    ax = ax.flatten()
    # define x and y limits
    xmin, xmax = AOI.geometry[0].buffer(500).bounds[0]/1e3, AOI.geometry[0].buffer(500).bounds[2]/1e3
    ymin, ymax = AOI.geometry[0].buffer(500).bounds[1]/1e3, AOI.geometry[0].buffer(500).bounds[3]/1e3
    # define colors for plotting
    color_snow = '#4eb3d3'
    color_ice = '#084081'
    color_rock = '#fdbb84'
    color_water = '#bdbdbd'
    # create colormap
    colors = [color_snow, color_snow, color_ice, color_rock, color_water]
    cmp = matplotlib.colors.ListedColormap(colors)
    # RGB image
    ax[0].imshow(np.dstack([im[ds_dict['RGB_bands'][0]].data,
                            im[ds_dict['RGB_bands'][1]].data,
                            im[ds_dict['RGB_bands'][2]].data]),
                 extent=(np.min(im.x.data)/1e3, np.max(im.x.data)/1e3, np.min(im.y.data)/1e3, np.max(im.y.data)/1e3))
    ax[0].set_xlabel('Easting [km]')
    ax[0].set_ylabel('Northing [km]')
    # classified image
    ax[1].imshow(im_classified['classified'].data, cmap=cmp, vmin=1, vmax=5,
                 extent=(np.min(im_classified.x.data)/1e3, np.max(im_classified.x.data)/1e3, np.min(im_classified.y.data)/1e3, np.max(im_classified.y.data)/1e3))
    # plot dummy points for legend
    ax[1].scatter(0, 0, color=color_snow, s=50, label='snow')
    ax[1].scatter(0, 0, color=color_ice, s=50, label='ice')
    ax[1].scatter(0, 0, color=color_rock, s=50, label='rock')
    ax[1].scatter(0, 0, color=color_water, s=50, label='water')
    ax[1].set_xlabel('Easting [km]')
    # AOI
    for j, geom in enumerate(AOI.geometry[0].boundary.geoms):
        if j==0:
            ax[0].plot([x/1e3 for x in geom.coords.xy[0]], [y/1e3 for y in geom.coords.xy[1]], '-k', linewidth=1, label='AOI')
        else:
            ax[0].plot([x/1e3 for x in geom.coords.xy[0]], [y/1e3 for y in geom.coords.xy[1]], '-k', linewidth=1, label='_nolegend_')
        ax[1].plot([x/1e3 for x in geom.coords.xy[0]], [y/1e3 for y in geom.coords.xy[1]], '-k', linewidth=1, label='_nolegend_')
    # reset x and y limits
    ax[0].set_xlim(xmin, xmax)
    ax[0].set_ylim(ymin, ymax)
    ax[1].set_xlim(xmin, xmax)
    ax[1].set_ylim(ymin, ymax)
    # image bands histogram
    h_b = ax[2].hist(im[ds_dict['RGB_bands'][0]].data.flatten(), color='blue', histtype='step', linewidth=2, bins=100, label="blue")
    h_g = ax[2].hist(im[ds_dict['RGB_bands'][1]].data.flatten(), color='green', histtype='step', linewidth=2, bins=100, label="green")
    h_r = ax[2].hist(im[ds_dict['RGB_bands'][2]].data.flatten(), color='red', histtype='step', linewidth=2, bins=100, label="red")
    ax[2].set_xlabel("Surface reflectance")
    ax[2].set_ylabel("Pixel counts")
    ax[2].legend(loc='best')
    ax[2].grid()
    # normalized snow elevations histogram
    ax[3].bar(bin_centers, H_snow_est_elev_norm, width=(bin_centers[1]-bin_centers[0]), color=color_snow, align='center')
    ax[3].set_xlabel("Elevation [m]")
    ax[3].set_ylabel("% snow-covered")
    ax[3].grid()
    ax[3].set_xlim(elev_min-10, elev_max+10)
    ax[3].set_ylim(0,1)
    # plot estimated snow line coordinates
    ax[0].plot([x/1e3 for x in snowline_df['snowlines_coords'][0].coords.xy[0]],
               [y/1e3 for y in snowline_df['snowlines_coords'][0].coords.xy[1]], 
               '.', color='#f768a1', markersize=5, label='sl$_{estimated}$')
    ax[1].plot([x/1e3 for x in snowline_df['snowlines_coords'][0].coords.xy[0]],
               [y/1e3 for y in snowline_df['snowlines_coords'][0].coords.xy[1]], 
               '.', color='#f768a1', markersize=5, label='sl$_{estimated}$')   
    # determine figure title and file name
    title = im_date + '_' + site_name + '_' + dataset + '_snowline'
    # add legends
    ax[0].legend(loc='best')
    ax[1].legend(loc='best')
    fig.suptitle(title)
    fig.tight_layout()
    # save figure
    fig_fn = figures_out_path + title + '.png'
    fig.savefig(fig_fn, dpi=300, facecolor='white', edgecolor='none')
    print('Figure saved to file:' + fig_fn)

    return fig
    

In [ ]:
import shapely 

# loop through images
for i, im_mosaic_fn in enumerate(im_mosaic_fns):
    print(' ')
    print(str(i+1)+'/'+str(len(im_mosaic_fns)))

    # adjust radiometry
    im_adj, im_adj_method = f.PS_adjust_image_radiometry(im_mosaic_fn, PS_im_mosaics_path, polygon_top, 
                                                         polygon_bottom, AOI_UTM, dataset_dict, dataset, 
                                                         site_name, skip_clipped, plot_results)
    if type(im_adj)==str: # skip if there was an error in adjustment
        continue
    
    # -----Determiine image date
    im_date = im_mosaic_fn[0:8] + 'T' + im_mosaic_fn[9:11] + ':00:00'
    
    # -----Classify image
    im_classified_fn = im_date.replace('-','').replace(':','') + '_' + site_name + '_' + dataset + '_classified.nc'
    if os.path.exists(im_classified_path + im_classified_fn):
        print('Classified image already exists in file, loading...')
        im_classified = xr.open_dataset(im_classified_path + im_classified_fn)
    else:
        im_classified = 'N/A'

    if type(im_classified)==str:
        continue    
    
    # -----Delineate snowline(s)
    snowline_fn = im_date.replace('-','').replace(':','') + '_' + site_name + '_' + dataset + '_snowline.pkl'
    if os.path.exists(snowlines_path + snowline_fn):
        print('Snowline already exists in file, loading...')
        snowline_df = pd.read_pickle(snowlines_path + snowline_fn)
        if type(snowline_df['snowlines_coords'][0])==shapely.geometry.linestring.LineString:
            fig = plot_snowline_image(im_adj, im_classified, snowline_df, AOI, DEM, site_name, im_date, 
                                      figures_out_path, dataset, dataset_dict)   
    else:
        continue